In [108]:
# libraries to use
import requests
import pandas as pd
import json
import io
import itertools
import datetime
import numpy as np
import math

In [109]:
# get game logs for PlayerID from NBA.com (MeasureType=Base,Advanced,Misc,Scoring,Usage)
def get_game_logs_df(PlayerID, MeasureType, Season, PerMode):
    glogs_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType='+MeasureType+'&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerID='+str(PlayerID)+'&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
    glogs_response = requests.get(url=glogs_url, headers=headers).json()
    glogs_headers = glogs_response['resultSets'][0]['headers']
    glogs_data = glogs_response['resultSets'][0]['rowSet']
    glogs_df = pd.DataFrame(data=glogs_data, columns=glogs_headers)
    return glogs_df

In [110]:
def get_games_past_week(this_game, plyr_box_stats_df):
    game_date = this_game['GAME_DATE_OBJ']
    # for a given game_date find the week period before
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = plyr_box_stats_df.loc[(plyr_box_stats_df['GAME_DATE_OBJ'] <= date_yesterday) & (plyr_box_stats_df['GAME_DATE_OBJ'] >= date_week_ago)]
    return game_dates_past_week

In [154]:
# to-do: support for multiple teams for the case player is traded mid-season
# (exmpale: James Harden 2021-22 traded BRK -> PHI)

def calc_injury_stats(plyr_box_stats_df, team, last_injury_date, num_career_injuries):
    player_games = plyr_box_stats_df.loc[plyr_box_stats_df['TEAM_ABBREVIATION']==team]
    # reverse player_games since team schedule first -> last
    player_games.sort_values(by='GAME_DATE_OBJ', ascending=True, inplace = True)
    player_dates = player_games['GAME_DATE_OBJ']
    # to-do: use for later (multi-team support)
    # max_date = max(player_dates)
    # min_date = min(player_dates)
    # global: team_dates_dict
    team_schedule = team_dates_dict.get(team)
    team_dates = team_schedule['dateGame_OBJ']
    # to-do: use for later (multi-team support)
    # team_max_date = max(team_dates)
    # team_min_date = min(team_dates)
    # get list of games in team schedule player wasn't in
    missed_dates = np.setdiff1d(team_dates,player_dates, assume_unique=False)
    played_schedule = []
    for date in team_dates:
        if date in missed_dates:
            played_schedule.append(False)
        else:
            played_schedule.append(True)
    
    # create temp table of team schedule with player in game
    temp_df = pd.DataFrame({'GAME_DATE_OBJ':team_dates,
                           'PLAYED_GAME':played_schedule,})
    temp_df.reset_index(drop=True, inplace=True)

    # create column flagging player out next game
    out_next_game = []
    for index, current_date in temp_df.iterrows():
        # if at last player's game of regular season just put true (data doesn't extend to playoffs)
        if index == len(temp_df) - 1:
            out_next_game.append(0)
        else:
            next_date = temp_df.iloc[index + 1]
            if next_date['PLAYED_GAME']:
                out_next_game.append(0)
            else:
                out_next_game.append(1)
    temp_df.loc[:,'OUT_NEXT_GAME']  = out_next_game

    # get OUT_NEXT_GAME subset based on player game logs
    player_injury_data = temp_df.loc[temp_df['GAME_DATE_OBJ'].isin(player_dates)]

    # calculate days last injury and rolling career injuries during season
    days_last_injury = []
    date_last_injury = last_injury_date
    career_injuries = []
    career_injuries_count = num_career_injuries
    for idx, row in player_injury_data.iterrows():
        if row['OUT_NEXT_GAME'] == 1:
            career_injuries_count = career_injuries_count + 1
            date_last_injury = row['GAME_DATE_OBJ']
    
        time_delta = row['GAME_DATE_OBJ'] - date_last_injury
        days_last_injury.append(time_delta.days)
        career_injuries.append(career_injuries_count)

    player_injury_data.loc[:,'CAREER_INJURIES'] = career_injuries
    player_injury_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury
        
    # reverse the injury list  because player game logs/plyr_box_stats_df last -> first
    player_injury_data = player_injury_data.iloc[::-1]
    player_injury_data.reset_index(drop=True, inplace=True)
    
    plyr_box_stats_df.loc[:,'OUT_NEXT_GAME'] = player_injury_data['OUT_NEXT_GAME']
    plyr_box_stats_df.loc[:,'CAREER_INJURIES'] = player_injury_data['CAREER_INJURIES']
    plyr_box_stats_df.loc[:,'DAYS_LAST_INJURY'] = player_injury_data['DAYS_LAST_INJURY']
    
    return plyr_box_stats_df


In [158]:
def get_full_game_logs_df(PlayerID, Season, PerMode):
    # get player game logs for PlayerID from NBA.com for all measure types (MeasureType=Base,Advanced,Misc,Scoring,Usage)
    # PlayerID = 1629029 # Luka Doncic
    # Season='2021-22'
    # PerMode='Totals'
    plyr_base_df = get_game_logs_df(PlayerID,'Base',Season, PerMode)
    plyr_advanced_df = get_game_logs_df(PlayerID,'Advanced',Season, PerMode)
    plyr_misc_df = get_game_logs_df(PlayerID,'Misc',Season, PerMode)
    plyr_scor_df = get_game_logs_df(PlayerID,'Scoring',Season, PerMode)
    plyr_usage_df = get_game_logs_df(PlayerID,'Usage',Season, PerMode)

    # get subset of attributes from each game log measure
    plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset]
    plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
    plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
    plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]

    # put all game log subsets for player into one dataframe
    plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                                   plyr_advanced_df_subset, 
                                   plyr_misc_df_subset, 
                                   plyr_scor_df_subset], axis=1)

    num_box_stats = plyr_box_stats_df.shape[0]

    # Add Bio data to game log dataframe: Age, Height (in.), Weight (lbs.)
    # global: plyrs_bio_df
    player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == PlayerID]

    # cannot find player birthday info on NBA.com
    # NOTE: NBA.com provides appropriate age in bio given Season
    player_age = [int(player_bio.iloc[0]['AGE'])] * num_box_stats
    plyr_box_stats_df['AGE'] = player_age

    player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
    plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

    player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
    plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

    # calculate field goals made mid-range and in the paint
    fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

    fgm_paint_list = []
    fgm_mr_list = []
    for x in range(0, num_box_stats):
        game_stats = plyr_box_stats_df.iloc[x][fg_attr]
        fgm_paint = game_stats['PTS_PAINT']/2
        fgm_2p = game_stats['FGM'] - game_stats['FG3M']
        fgm_mr = fgm_2p - fgm_paint
        fgm_paint_list.append(int(fgm_paint))
        fgm_mr_list.append(int(fgm_mr))

    # note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
    # will use 2PT mid-range and in-the-paint made FG for now
    plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
    plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list

    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]
    # add converted python GAME_DATE_OBJ datetime64 object to plyr_box_stats_df
    plyr_box_stats_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj

    # Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
    # Total minutes played - pw_min
    # Total games played - pw_games
    # 2-point field goal attempts - pw_fg2a
    # 3-point field goal attempts - pw_gf3a
    # Blocks - pw_blk
    # Rebounds - pw_reb (need offensive/defensive rebounds…?)
    # Personal fouls - pw_pf
    # Assists - pw_ast
    # Other - ???

    pw_mins = []
    pw_gamess = []
    pw_fg3as = []
    pw_fg2as = []
    pw_orebs = []
    pw_drebs = []
    pw_asts = []
    pw_tovs = []
    pw_stls = []
    pw_blks = []
    pw_posss = []
    pw_pfs = []
    pw_pfds = []
    pw_fgm_paints = []
    pw_fgm_2pt_mrs = []
    pw_pie_avg = []
    pw_usg_pct_avg = []

    # iterate through all games in player box stats and calculate totals/avgs for games the week before
    for index, this_game in plyr_box_stats_df.iterrows():
        games_week_before_df = get_games_past_week(this_game, plyr_box_stats_df)
        pw_min = games_week_before_df['MIN'].sum()
        pw_games = games_week_before_df.shape[0]
        pw_fg3a = games_week_before_df['FG3A'].sum()
        pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
        pw_oreb = games_week_before_df['OREB'].sum()
        pw_dreb = games_week_before_df['DREB'].sum()
        pw_ast = games_week_before_df['AST'].sum()
        pw_tov = games_week_before_df['TOV'].sum()
        pw_stl = games_week_before_df['STL'].sum()
        pw_blk = games_week_before_df['BLK'].sum()
        pw_poss = games_week_before_df['POSS'].sum()
        pw_pf = games_week_before_df['PF'].sum()
        pw_pfd = games_week_before_df['PFD'].sum()
        pw_fgm_paint = games_week_before_df['PTS_PAINT'].sum()/2
        pw_fgm_2p = games_week_before_df['FGM'].sum() - games_week_before_df['FG3M'].sum()
        pw_fgm_2pt_mr = pw_fgm_2p - pw_fgm_paint
        pw_pie = games_week_before_df['PIE'].mean()
        pw_usg_pct = games_week_before_df['USG_PCT'].mean()

        pw_mins.append(pw_min)
        pw_gamess.append(pw_games)
        pw_fg3as.append(pw_fg3a)
        pw_fg2as.append(pw_fg2a)
        pw_orebs.append(pw_oreb)
        pw_drebs.append(pw_dreb)
        pw_asts.append(pw_ast)
        pw_tovs.append(pw_tov)
        pw_stls.append(pw_stl)
        pw_blks.append(pw_blk)
        pw_posss.append(pw_poss)
        pw_pfs.append(pw_pf)
        pw_pfds.append(pw_pfd)
        pw_fgm_paints.append(pw_fgm_paint)
        pw_fgm_2pt_mrs.append(pw_fgm_2pt_mr)
        pw_pie_avg.append(pw_pie)
        pw_usg_pct_avg.append(pw_usg_pct)

    plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
    plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
    plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
    plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
    plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
    plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
    plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
    plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
    plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
    plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
    plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
    plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
    plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
    plyr_box_stats_df.loc[:,'PW_FGM_PAINT'] = pw_fgm_paints
    plyr_box_stats_df.loc[:,'PW_FGM_2PT_MR'] = pw_fgm_2pt_mrs
    plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
    plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

    # fill na with 0 on past week stats
    #plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
    plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
    plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

    # get player injury data
    # golbal: il_data, mg_data
    player_il = il_data.loc[il_data['PLAYER_ID'] == PlayerID]
    player_mg = mg_data.loc[mg_data['PLAYER_ID'] == PlayerID]

    # get first date game played for season (at end of list becasue last -> first)
    first_game_season = plyr_box_stats_df.iloc[len(plyr_box_stats_df) - 1]
    first_game_season_date = first_game_season['GAME_DATE_OBJ']

    # get missed game history before season
    player_mg_before_season = player_mg.loc[player_mg['Date_OBJ'] < first_game_season_date]
    player_mg_before_season = player_mg_before_season.loc[pd.isnull(player_mg_before_season['Acquired'])]

    # find date of last missed game before season
    last_mg = player_mg_before_season.iloc[len(player_mg_before_season) - 1]
    last_mg_date = last_mg['Date_OBJ']

    # get injured list history before season
    player_il_before_season = player_il.loc[player_il['Date_OBJ'] < first_game_season_date]
    player_il_before_season = player_il_before_season.loc[pd.isnull(player_il_before_season['Acquired'])]

    # find date of last missed game before season
    last_il = player_il_before_season.iloc[len(player_il_before_season) - 1]
    last_il_date = last_il['Date_OBJ']

    # get last injury or missed date before season
    # will use as inital 'anchor date' to calculate days from last injury
    last_injury_date = max(last_il_date, last_mg_date)

    # get number of career injuries before season
    # will use as inital count to calculate rolling injuries during season
    num_career_injuries = len(player_mg_before_season) + len(player_il_before_season)

    # get list of teams player was on during season (if > 1 -> was traded mid-season)
    player_teams = plyr_box_stats_df['TEAM_ABBREVIATION'].unique()

    # injury stats can only support a player on the same team thoughout the season (i.e. not traded mid-season)
    if len(player_teams) == 1:
        plyr_box_stats_df = calc_injury_stats(plyr_box_stats_df, player_teams[0], last_injury_date, num_career_injuries)
    else:
        print('Skipping player ' + plyr_box_stats_df['PLAYER_NAME'][0] + ' injury data who was traded mid-season (to-do)')

    # save game log stats for player to .csv file
    player_name = plyr_box_stats_df['PLAYER_NAME'][0]
    player_name = player_name.replace(" ", "_")
    file_name = 'data/2021_22_NBA_REG_GAME_LOGS_' + player_name + '_' + str(plyr_box_stats_df['PLAYER_ID'][0]) + '.csv'
    plyr_box_stats_df.to_csv(file_name)
    print('Write to file: ' + file_name)





In [113]:
# use these headers for every request to NBA.com
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [114]:
Season='2021-22'
PerMode='Totals'

# make a list of NBA measure types to loop through for a player
MeasureTypes = [
    'Base',
    'Advanced',
    'Misc',
    'Scoring',
    'Usage'
]

# make a list of NBA seasons to loop through later
season_list = [
    '1996-97',
    '1997-98',
    '1998-99',
    '1999-00',
    '2000-01',
    '2001-02',
    '2002-03',
    '2003-04',
    '2004-05',
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2011-12',
    '2012-13',
    '2013-14',
    '2014-15',
    '2015-16',
    '2016-17',
    '2017-18',
    '2018-19',
    '2019-20',
    '2020-21',
    '2021-22',
    '2022-23',
]

team_abbrev = ['BOS', 'GSW', 'DET', 'WAS', 'ATL', 'NOP', 'CHI', 'TOR', 'MEM',
       'MIN', 'CHA', 'UTA', 'PHX', 'POR', 'MIL', 'LAC', 'SAS', 'BKN',
       'NYK', 'DEN', 'IND', 'MIA', 'CLE', 'DAL', 'PHI', 'HOU', 'OKC',
       'ORL', 'SAC', 'LAL']

In [115]:
# get general (Measure=Base) player total stats (PerMode=Totals) for 2021-22 season (Season=2021-22) - example URL request
# get PLAYER_ID, PLAYER_NAME, GP, MIN
players_stats_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode='+PerMode+'&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+Season+'&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
players_stats_response = requests.get(url=players_stats_url, headers=headers).json()
players_stats_headers = players_stats_response['resultSets'][0]['headers']
players_stats_data = players_stats_response['resultSets'][0]['rowSet']
players_stats_df = pd.DataFrame(data=players_stats_data, columns=players_stats_headers)

In [117]:
# get subset of players_stats_df containing only PLAYER_ID, PLAYER_NAME, MIN
player_ids_2021_22 = players_stats_df[['PLAYER_ID', 'PLAYER_NAME','GP','MIN']]

# using stat Totals from NBA.com calculate MPG
mpg_list = []
for x in range(0, player_ids_2021_22.shape[0]):
    mpg = player_ids_2021_22.iloc[x]['MIN']/player_ids_2021_22.iloc[x]['GP']
    mpg_list.append(round(mpg,1))

player_ids_2021_22.loc[:,'MPG'] = mpg_list

C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\3826829581.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_ids_2021_22.loc[:,'MPG'] = mpg_list


In [118]:
# limit study to only players with 30MPG in the 2021-22 NBA Regular Season
high_mpg_players_2021_22_df = player_ids_2021_22.loc[(player_ids_2021_22['MIN'] >= 500) & (player_ids_2021_22['MPG'] >= 30)]
# save this data to a file for reference
high_mpg_players_2021_22_df.to_csv('players_mpg_2021_22_30mpg.csv', index=False)

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,203932,Aaron Gordon,75,2375.418333,31.7
21,203952,Andrew Wiggins,73,2329.513333,31.9
23,203076,Anthony Davis,40,1404.248333,35.1
24,1630162,Anthony Edwards,72,2465.993333,34.2
34,1628389,Bam Adebayo,56,1825.261667,32.6
...,...,...,...,...,...
571,1629639,Tyler Herro,66,2151.011667,32.6
574,1630169,Tyrese Haliburton,77,2694.131667,35.0
575,1630178,Tyrese Maxey,75,2650.356667,35.3
591,203115,Will Barton,71,2276.463333,32.1


In [119]:
# get 2021-22 NBA Season schedule based on nbastatR::seasons_schedule()
schedule_df = pd.read_csv("schedule_2021_22.csv")

# convert dateGame string into datetime64 object, add it to schedule dataframe
sch_game_dates = schedule_df['dateGame'].values.tolist()
sch_game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in sch_game_dates]
schedule_df.loc[:,'dateGame_OBJ']  = sch_game_dates_obj

# create a dictionary of team to list of games played 
team_dates_dict = {}
for team in team_abbrev:
    team_games = schedule_df.loc[(schedule_df['slugTeamWinner'] == team) | (schedule_df['slugTeamLoser'] == team)]
    team_dates_dict[team]=team_games


In [120]:
# load player dictionary based on data from nbastatR::players_careers()
player_dict = pd.read_csv('df_nba_player_dict.csv')

In [121]:
# load all injured list data since 2010 from Pro Sports Transactions
il_data = pd.read_csv('prosportstransactions_scrape_inactivelist_2010_2023.csv')
# convert Date string into datetime64 object, add it to il dataframe
il_dates = il_data['Date'].values.tolist()
il_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in il_dates]
il_data.loc[:,'Date_OBJ']  = il_dates_obj

# if possible map NBA Player IDs to entires in injured list
il_id = []

num_cannot_find = 0

for index, row in il_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        il_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        il_id.append(int(round(player['idPlayer'].values[0],0)))

print('cannot find: ' + str(num_cannot_find))

il_data.loc[:,'PLAYER_ID'] = il_id

cannot find: Matt Janning
cannot find: Maurice Williams / Mo Williams
cannot find: Alexis Ajinca / Alex Ajinca
cannot find: A.J. Price
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Louis Amundson / Lou Amundson
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Williams / Mo Williams
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Patrick Mills / Patty Mills
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Ishmael Smith / Ish Smith
cannot find: Eugene Jeter / Pooh Jeter
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Maurice Williams / Mo Williams
cannot find: Marcus Thornton (T.)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Joe Smith (Leynard)
cannot find: Ed Davis (a)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Gerald 

cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Greg Smith (a)
cannot find: Patrick Mills / Patty Mills
cannot find: Malcolm Thomas (b)
cannot find: Bill Walker
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Chris Wright (b)
cannot find: Maurice Williams / Mo Williams
cannot find: Chris Johnson (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Chris Johnson (a)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: C.J. Miles
cannot find: (William) Tony Parker
cannot find: Hidayet Turkoglu / Hedo Turkoglu
cannot find: Walker Russell Jr.
cannot find: Courtney Simpson
cannot find: Greg Smith (a)
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: 

cannot find: J.J. Redick
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: A.J. Price
cannot find: Byron Mullens / B.J. Mullens
cannot find: Tony Mitchell (b)
cannot find: A.J. Price
cannot find: Ray McCallum Jr.
cannot find: Malcolm Thomas (b)
cannot find: Ray McCallum Jr.
cannot find: Byron Mullens / B.J. Mullens
cannot find: Metta World Peace / Ron Artest
cannot find: A.J. Price
cannot find: James Jones (b)
cannot find: Byron Mullens / B.J. Mullens
cannot find: A.J. Price
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: A.J. Price
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Mike Conley Jr.
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Tony Mitchell (b)
cannot find: Byron Mullens / B.J. Mullens
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: J.J. Redick
cannot

cannot find: John Wall (a)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: John Wall (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Greg Smith (a)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: Mike Conley Jr.
cannot find: Greg Smith (a)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Mike Conley Jr.
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Mike Dunleavy Jr.
cannot find: Walter Tavares / Edy Tavares
cannot find: Norris Cole (a)
cannot find: Tony Wroten Jr.
cannot find: Gerald Henderson (b)
cannot find: R.J. Hunter
cannot fi

cannot find: Stephen Zimmerman Jr.
cannot find: R.J. Hunter
cannot find: Marcelinho Huertas / Marcelo Huertas
cannot find: Stephen Zimmerman Jr.
cannot find: John Wall (a)
cannot find: Danuel House
cannot find: Kahlil Felder / Kay Felder
cannot find: J.R. Smith
cannot find: Metta World Peace / Ron Artest
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Metta World Peace / Ron Artest
cannot find: Wesley Johnson / Wes Johnson
cannot find: C.J. Miles
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Metta World Peace / Ron Artest
cannot find: Stephen Zimmerman Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Kahlil Felder / Kay Felder
cannot find: Stephen Zimmerman Jr.
cannot find: James Michael McAdoo / James McAdoo (Michael)
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: James Michael McAdoo 

cannot find: Raulzinho Neto / Raul Neto
cannot find: Stephen Zimmerman Jr.
cannot find: Sheldon McClellan / Sheldon Mac
cannot find: Raulzinho Neto / Raul Neto
cannot find: Kahlil Felder / Kay Felder
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Kahlil Felder / Kay Felder
cannot find: James Ennis
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Stephen Zimmerman Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Sheldon McClellan / Sheldon Mac
cannot find: John Wall (a)
cannot find: Kahlil Felder / Kay Felder
cannot find: Walter Tavares / Edy Tavares
cannot find: Raulzinho Neto / Raul Neto
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Raulzinho Neto / Raul Neto
cannot find: Norris Cole (a)
cannot find: Norris Cole (a)
cannot find: James Ennis
cannot find: (William) Tony Parker
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: Marc

cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: C.J. Miles
cannot find: Marcus Morris
cannot find: C.J. Miles
cannot find: R.J. Hunter
cannot find: Zhou Qi / Qi Zhou
cannot find: Marcus Morris
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Danuel House
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Maurice Harkless / Moe Harkless
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: R.J. Hunter
cannot find: Zhou Qi / Qi Zhou
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Marcus Morris
cannot find: Enes Kanter
cannot find: John Wall (a)
cannot find: Ed Davis (a)
cannot find: Georgios Papagiannis / Giorgios Papagiannis / George Papagiannis
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Wayne Selden 

cannot find: Frank Mason
cannot find: Frank Mason
cannot find: Moritz Wagner / Moe Wagner
cannot find: Mike Conley Jr.
cannot find: Mike Conley Jr.
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Frank Mason
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Frank Mason
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Mohamed Bamba / Mo Bamba
cannot find: 11/25/2019
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Cameron Reddish / Cam Reddish
cannot find: Cameron Reddish / Cam Reddish
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Marcus Morris
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Morris
cannot find: Maurice Harkless / Moe Harkless
cannot find: Mike Conley Jr.
cannot find: Maurice Harkless / Moe Harkless
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Enes Kanter / Enes Freedom
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Mike Conley Jr.
c

cannot find: Ishmael Smith / Ish Smith
cannot find: Ignas Brazdeikis / Iggy Brazdeikis
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Moritz Wagner / Moe Wagner
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: Marcus Morris
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: J.T. Thor
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Ed Davis (Adam)
cannot find: Daulton Hommes
cannot find: Louis Williams / Lou Williams
cannot find: Enes Freedom / Enes Kanter
cannot find: Cameron Thomas / Cam Thomas
cannot find: J.T. Thor
cannot find: C.J. Miles
cannot find: Justin Jackson (Aaron)
cannot find: Wesley Iwundu / Wes Iwundu
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Juancho Hernangomez / Juan Hernangomez
cannot find: Ruben Nembhard / R.J. Nembhard Jr.
cannot find: B.J. Johnson
cannot find: D.J. Augustine


In [122]:
# load all missed games data since 2010 from Pro Sports Transactions
mg_data = pd.read_csv('prosportstransactions_scrape_missedgames_2010_2023.csv')
# convert Date string into datetime64 object, add it to mg dataframe
mg_dates = mg_data['Date'].values.tolist()
mg_dates_obj = [datetime.datetime.strptime(date, '%m/%d/%Y') for date in mg_dates]
mg_data.loc[:,'Date_OBJ']  = mg_dates_obj

# if possible map NBA Player IDs to entires in missed games list

mg_id = []

num_cannot_find = 0

for index, row in mg_data.iterrows():
    namePlayer = row['Acquired']
    if pd.isnull(row['Acquired']):
        namePlayer = row['Relinquished']
    player = player_dict.loc[player_dict['namePlayer'] == namePlayer]
    if (player.empty):
        mg_id.append(np.nan)
        print('cannot find: ' + str(namePlayer))
        num_cannot_find = num_cannot_find + 1
    else:
        mg_id.append(round(player['idPlayer'].values[0],0))
        
print('cannot find: ' + str(num_cannot_find))

mg_data.loc[:,'PLAYER_ID'] = mg_id

cannot find: Ed Davis (a)
cannot find: Jeff Pendergraph / Jeff Ayres
cannot find: Louis Amundson / Lou Amundson
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Mike Dunleavy Jr.
cannot find: Mike Dunleavy Jr.
cannot find: John Wall (a)
cannot find: Marcus Thornton (T.)
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: J.J. Redick
cannot find: J.J. Redick
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: John Wall (a)
cannot find: Zaur Pachulia / Zaza Pachulia
cannot find: D.J. White
cannot find: Predrag Stojakovic / Peja Stojakovic
cannot find: Predrag Stojakovi

cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: C.J. Miles
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: C.J. Miles
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: D.J. Augustine
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: D.J. Augustine
cannot find: Larry Sanders (b)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Marcus Thornton (T.)
cannot find: C.J. Miles
cannot find: D.J. Augustine
cannot find: Malcolm Lee / Malcom Lee
cannot find: Marcus Thornton (T.)
cannot find: Louis Williams / Lou Williams
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Louis Williams / Lou Williams
cannot find: J.J. Redick
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Amare Stoudemire / Amar'e Stoudemire
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: James Jones (b)
cannot find: John Wall (a)
cannot find: Chris Smith (c)
cannot find: Amare Stoudemire / Amar'e Stoudemi

cannot find: Metta World Peace / Ron Artest
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Greg Smith (a)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Greg Smith (a)
cannot find: Greg Smith (a)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Wesley Johnson / Wes Johnson
cannot find: Greg Smith (a)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Metta World Peace / Ron Artest
cannot find: Metta World Peace / Ron Artest
cannot find: Jeffery Taylor / Jeff Taylor (b)
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Metta World Peace / Ron Artest
cannot find: Tony Wroten Jr.
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Tony Wroten Jr.
cannot find: Jeffery Taylor / Jeff

cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Louis Amundson / Lou Amundson
cannot find: Louis Amundson / Lou Amundson
cannot find: Mike Conley Jr.
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Emanuel Ginobili / Manu Ginobili
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Justin Hamilton (b)
cannot find: Ognjen Kuzmic / Ognen Kuzmic
cannot find: Marcus Thornton (T.)
cannot find: Jeff Ayres / Jeff Pendergraph
cannot find: Enes Kanter
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Enes Kanter
cannot find: Jeff Ayres / Jeff Pendergraph
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cannot find: Justin Hamilton (b)
cannot find: Toure' Murry / Toure Murry
cannot find: Toure' Murry / Toure Murry
cannot find: Luc Richard Mbah a Moute / Luc Mbah a Moute
cann

cannot find: Mike Conley Jr.
cannot find: J.J. Redick
cannot find: Marcus Morris
cannot find: Maurice Ndour / Maurice N'dour
cannot find: Mike Dunleavy Jr.
cannot find: Mike Conley Jr.
cannot find: Maurice Ndour / Maurice N'dour
cannot find: Marcus Morris
cannot find: Mike Dunleavy Jr.
cannot find: Gerald Henderson (b)
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Maurice Harkless / Moe Harkless
cannot find: Patrick Mills / Patty Mills
cannot find: Maurice Harkless / Moe Harkless
cannot find: Wesley Johnson / Wes Johnson
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Ed Davis (a)
cannot find: Patrick Mills / Patty Mills
cannot find: DeAndre' Bembry / DeAndre Bembry
cannot find: Wesley Johnson / Wes Johnson
cannot find: (William) Tony Parker
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Ed Davis (a)
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Ema

cannot find: James Ennis
cannot find: Moritz Wagner / Moe Wagner
cannot find: Maurice Harkless / Moe Harkless
cannot find: Zhou Qi / Qi Zhou
cannot find: Kevin Knox
cannot find: C.J. Miles
cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: J.R. Smith
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: Jose Juan Barea / Jose Barea / J.J. Barea
cannot find: Guillermo Hernangomez / Willy Hernangomez
cannot find: (William) Tony Parker
cannot find: (William) Tony Parker
cannot find: Gerald Green (b)
cannot find: Zhou Qi / Qi Zhou
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Gerald Green (b)
cannot find: Nene / Nene Hilario / Maybyner Hilario
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: Wayne Selden Jr.
cannot find: John Wall (a)
cannot find: John Wall (a)
cannot find: Louis Williams / Lou Williams
cannot find: John Wall 

cannot find: (Sean) Chris Smith
cannot find: Raulzinho Neto / Raul Neto
cannot find: K.Z. Okpala
cannot find: K.Z. Okpala
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Maurice Harkless / Moe Harkless
cannot find: Mohamed Bamba / Mo Bamba
cannot find: nan
cannot find: Mohamed Bamba / Mo Bamba
cannot find: Ishmail Wainright / Ish Wainright
cannot find: Maurice Harkless / Moe Harkless
cannot find: Maurice Harkless / Moe Harkless
cannot find: (Sean) Chris Smith
cannot find: Marcus Morris
cannot find: Marcus Morris
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Nah'Shon Hyland / Bones Hyland
cannot find: Marcus Morris
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Marcus Morris
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Maximilian Kleber / Maxi Kleber
cannot find: Nicolas Claxton / Nic Claxton
cannot find: Wesley Matthews / Wes Matthews Jr.
cannot find: Wesley Matthew

In [151]:
# example NBA.com 2021-21 (Season=2021-22) players bio (https://stats.nba.com/stats/leaguedashplayerbiostats):
# use for AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT
plyrs_bio_url = 'https://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season='+Season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
plyrs_bio_response = requests.get(url=plyrs_bio_url, headers=headers).json()
plyrs_bio_headers = plyrs_bio_response['resultSets'][0]['headers']
plyrs_bio_data = plyrs_bio_response['resultSets'][0]['rowSet']
plyrs_bio_df = pd.DataFrame(data=plyrs_bio_data, columns=plyrs_bio_headers)

In [148]:
# example NBA.com 2021-21 (Season=2021-22) player index (https://stats.nba.com/stats/playergamelogs/playerindex):
# do we need anything...? Just get from bio?
plyrs_idx_url = 'https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season='+Season+'&SeasonType=Regular%20Season&TeamID=0&Weight='
plyrs_idx_response = requests.get(url=plyrs_idx_url, headers=headers).json()
plyrs_idx_headers = plyrs_idx_response['resultSets'][0]['headers']
plyrs_idx_data = plyrs_idx_response['resultSets'][0]['rowSet']
plyrs_idx_df = pd.DataFrame(data=plyrs_idx_data, columns=plyrs_idx_headers)
plyrs_idx_headers

['PERSON_ID',
 'PLAYER_LAST_NAME',
 'PLAYER_FIRST_NAME',
 'PLAYER_SLUG',
 'TEAM_ID',
 'TEAM_SLUG',
 'IS_DEFUNCT',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'JERSEY_NUMBER',
 'POSITION',
 'HEIGHT',
 'WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'ROSTER_STATUS',
 'PTS',
 'REB',
 'AST',
 'STATS_TIMEFRAME',
 'FROM_YEAR',
 'TO_YEAR']

In [125]:
# selection subset of attributes from each game log measure

plyr_base_headers_subset = ['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PFD',
 'PTS']

plyr_advanced_headers_subset = ['USG_PCT',
 'PIE',
 'POSS']

plyr_misc_headers_subset= [ 'PTS_PAINT']

plyr_scor_headers_subset = ['PCT_PTS_2PT_MR']

In [159]:
# Test with misc PlayerIDs
PlayerID = 2544 # leBron James
# PlayerID = 202695 # kawhi leonard - lots of time off - no bio?! out all of 2021-22 season, no bio for season - DON'T USE!
# PlayerID = 201935 # James Harden - had 2021-22 trade BRK -> PHI
# PlayerID = 1629029 # Luka Doncic
get_full_game_logs_df(PlayerID, Season, PerMode)

Write to file: data/2021_22_NBA_REG_GAME_LOGS_LeBron_James_2544.csv


C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\1516442695.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_injury_data.loc[:,'CAREER_INJURIES'] = career_injuries
C:\Users\Reed\AppData\Local\Temp\ipykernel_29836\1516442695.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_injury_data.loc[:,'DAYS_LAST_INJURY'] = days_last_injury


In [ ]:
# loop through PLAYER_ID in high_mpg_players_2021_22_df to generate .csv game logs
PlayerIDs = high_mpg_players_2021_22_df['PLAYER_ID'].values.tolist()

for PlayerID in PlayerIDs:
    get_full_game_logs_df(PlayerID, MeasureType, Season, PerMode)
